In [148]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib ipympl

Jan_return = pd.read_csv('Jan_return.csv')
Aug_return = pd.read_csv('Aug_return.csv')
Oct_return = pd.read_csv('Oct_return.csv')
Jan_return = Jan_return.drop(['Unnamed: 0'], axis=1)
Aug_return = Aug_return.drop(['Unnamed: 0'], axis=1)
Oct_return = Oct_return.drop(['Unnamed: 0'], axis=1)
Jan_predict_return = pd.read_csv('table_predict_return_Jan.csv')
Aug_predict_return = pd.read_csv('table_predict_return_Aug.csv')
Oct_predict_return = pd.read_csv('table_predict_return_Oct.csv')
Jan_predict_return = Jan_predict_return.drop(['Unnamed: 0'], axis=1)
Aug_predict_return = Aug_predict_return.drop(['Unnamed: 0'], axis=1)
Oct_predict_return = Oct_predict_return.drop(['Unnamed: 0'], axis=1)

"""replace returns with abs moret han 0.5 by 0"""
Jan_return[Jan_return > 0.5] = 0.01
Aug_return[Aug_return > 0.5] = 0.01
Oct_return[Oct_return > 0.5] = 0.01
Jan_return[Jan_return < -0.5] = 0.01
Aug_return[Aug_return < -0.5] = 0.01
Oct_return[Oct_return < -0.5] = 0.01


In [149]:

x = Jan_return.cov().to_numpy()
y = Aug_return.cov().to_numpy() 
z = Oct_return.cov().to_numpy() 
a = np.matmul(np.matmul(Jan_return.to_numpy(), np.linalg.inv(x)), Jan_return.to_numpy().T)
b = np.matmul(np.matmul(Aug_return.to_numpy(), np.linalg.inv(y)), Aug_return.to_numpy().T)
c = np.matmul(np.matmul(Oct_return.to_numpy(), np.linalg.inv(z)), Oct_return.to_numpy().T)

"""get the diagonal of a, b, c"""

a = np.diag(a)
b = np.diag(b)
c = np.diag(c)


new_a = np.zeros((a.shape[0],))
new_b = np.zeros((b.shape[0],))
new_c = np.zeros((c.shape[0],))


In [150]:

date_a = np.linspace(1/12, 0, 671)
date_b = np.linspace(1/12, 0, 623)
date_c = np.linspace(1/12, 0, 647)
new_a = a[1:] * date_a
new_b = b[1:] * date_b
new_c = c[1:] * date_c

In [151]:
origin_a = a
origin_b = b
origin_c = c
a = np.exp(new_a)
b = np.exp(new_b)
c = np.exp(new_c)

In [152]:
gamma = 0.001

policy_mean_coeff_Jan = -1 * np.matmul(np.linalg.inv(x) / 671, Jan_return.to_numpy().T).T
policy_mean_coeff_Aug = -1 * np.matmul(np.linalg.inv(y) / 623, Aug_return.to_numpy().T).T
policy_mean_coeff_Oct = -1 * np.matmul(np.linalg.inv(z) / 647, Oct_return.to_numpy().T).T
policy_mean_coeff_Jan = policy_mean_coeff_Jan[1:]
policy_mean_coeff_Aug = policy_mean_coeff_Aug[1:]
policy_mean_coeff_Oct = policy_mean_coeff_Oct[1:]

policy_var_coeff_Jan = np.zeros((671, 9,9))
policy_var_coeff_Aug = np.zeros((623, 9,9))
policy_var_coeff_Oct = np.zeros((647, 9,9))


for i in range(671):
    policy_var_coeff_Jan[i] = 0.5 * gamma  * np.linalg.inv(x) * a[i]
for i in range(623):
    policy_var_coeff_Aug[i] = 0.5 * gamma  * np.linalg.inv(y) * b[i]
for i in range(647):
    policy_var_coeff_Oct[i] = 0.5 * gamma  * np.linalg.inv(z) * c[i]

x_0 = 100
z = 105
Jan_wealth = [x_0]
Aug_wealth = [x_0]
Oct_wealth = [x_0]
for i in range(671):
    holdings = np.random.multivariate_normal(policy_mean_coeff_Jan[i] * (Jan_wealth[-1] - (z * np.exp(origin_a[i]/12) - x_0) / (np.exp(origin_a[i] / 12) - 1)), policy_var_coeff_Jan[i])
    profits = np.dot(holdings, Jan_return.iloc[i, :].to_numpy())
    Jan_wealth.append(profits + Jan_wealth[-1])
        

for i in range(623):
    holdings = np.random.multivariate_normal(policy_mean_coeff_Aug[i] * (Aug_wealth[-1] - (z * np.exp(origin_b[i]/12) - x_0) / (np.exp(origin_b[i] / 12) - 1)) , policy_var_coeff_Aug[i])
    profits = np.dot(holdings, Aug_return.iloc[i, :].to_numpy())
    Aug_wealth.append(profits + Aug_wealth[-1])


for i in range(647):   #647
    holdings = np.random.multivariate_normal(policy_mean_coeff_Oct[i] * (Oct_wealth[-1] - (z * np.exp(origin_c[i]/12) - x_0) / (np.exp(origin_c[i] / 12) - 1)), policy_var_coeff_Oct[i])
    profit = np.dot(holdings, Oct_return.iloc[i, :].to_numpy())
    print(profit)
    Oct_wealth.append(profit + Oct_wealth[-1])
    


plt.figure(figsize=(10, 5))
plt.plot(Jan_wealth, label = 'Jan')
plt.show()
#plt.savefig('Jan_wealth_second.png')

plt.figure(figsize=(10, 5))
plt.plot(Aug_wealth, label='Aug')
plt.show()
#plt.savefig('Aug_wealth_second.png')


plt.figure(figsize=(10, 5))
plt.plot(Oct_wealth, label='Oct')
plt.show()
#plt.savefig('Oct_wealth_second.png')